In [1]:
from web3 import Web3
from web3.gas_strategies.rpc import rpc_gas_price_strategy
from uniswap import Uniswap
import time
from dotenv import load_dotenv
import os

In [2]:
class TokenTrader:
    def __init__(self, rpc_url, wallet_address, private_key, token_address, usdt_address, eth_address, price_thresholds, lot_size_tokens, min_sell_amount_tokens):
        self.rpc_url = rpc_url
        self.wallet_address = wallet_address
        self.private_key = private_key
        self.token_address = token_address
        self.usdt_address = usdt_address
        self.eth_address = eth_address
        self.price_thresholds = price_thresholds
        self.lot_size_tokens = lot_size_tokens
        self.min_sell_amount_tokens = min_sell_amount_tokens
        
        self.web3 = Web3(Web3.HTTPProvider(self.rpc_url))
        self.web3.eth.set_gas_price_strategy(rpc_gas_price_strategy)
        self.uniswap = Uniswap(address=self.wallet_address, private_key=self.private_key, version=2, web3=self.web3, provider=self.rpc_url)
        
        self.token_decimals = self.get_token_decimals(self.token_address)
        self.lot_size = self.lot_size_tokens * 10**self.token_decimals
        self.min_sell_amount = self.min_sell_amount_tokens * 10**self.token_decimals

    def get_token_decimals(self, token_address):
        token_contract = self.uniswap.get_token(token_address)
        return token_contract.decimals

    def get_token_balance(self, token_address):
        return self.uniswap.get_token_balance(token_address)

    def get_price_from_pool(self, input_token, output_token):
        input_token_decimals = self.get_token_decimals(input_token)
        try:
            price = self.uniswap.get_price_input(input_token, output_token, 10**input_token_decimals)
            return price
        except Exception as e:
            print(f"An error occurred while fetching price: {e}")
            return None

    def get_token_price_in_usdt(self):
        token_price_in_eth = self.get_price_from_pool(self.token_address, self.eth_address)
        eth_price_in_usdt = self.get_price_from_pool(self.eth_address, self.usdt_address)
        if token_price_in_eth is not None and eth_price_in_usdt is not None:
            return (token_price_in_eth / 10**18) * (eth_price_in_usdt / 10**6)
        return None

    def sell_token_for_base(self, token_address, base_address, amount):
        tx = self.uniswap.make_trade(token_address, base_address, amount)
        return tx

    def trade(self):
        sold_tokens = [0] * len(self.price_thresholds)
        sold_lots = 0

        while sold_lots < len(self.price_thresholds):
            try:
                balance = self.get_token_balance(self.token_address)
                print(f"Current token balance: {balance / 10**self.token_decimals} tokens")
                price_in_usdt = self.get_token_price_in_usdt()
                print(f"Current token price: {price_in_usdt} USDT")

                while sold_lots < len(self.price_thresholds) and balance > self.min_sell_amount and price_in_usdt >= self.price_thresholds[sold_lots]:
                    remaining_lot = self.lot_size - sold_tokens[sold_lots]
                    amount_to_sell = min(balance, remaining_lot)
                    if amount_to_sell < self.min_sell_amount:
                        break

                    # Realizar la venta
                    #print(f"Selling {amount_to_sell / 10**self.token_decimals} tokens for {self.eth_address} at {price_in_usdt} USDT equivalent")
                    #tx = self.sell_token_for_base(self.token_address, self.eth_address, amount_to_sell)
                    #print(f"Transaction successful: {tx}")

                    sold_tokens[sold_lots] += amount_to_sell
                    balance -= amount_to_sell

                    if sold_tokens[sold_lots] >= self.lot_size:
                        sold_lots += 1

                time.sleep(2)
            except Exception as e:
                print(f"An error occurred: {e}")
                time.sleep(2)


In [ ]:
if __name__ == "__main__":
    # Cargar variables de entorno
    load_dotenv()
    
    # Configuración
    rpc_url = "https://rpc.ankr.com/eth" 
    wallet_address = os.getenv("WALLET_ADDRESS")
    private_key = os.getenv("PRIVATE_KEY")
    token_address = "0x8290333ceF9e6D528dD5618Fb97a76f268f3EDD4"
    usdt_address = "0xdAC17F958D2ee523a2206206994597C13D831ec7"
    eth_address = "0x0000000000000000000000000000000000000000"
    price_thresholds = [1.0, 1.5, 2.0, 2.5, 3.0]
    lot_size_tokens = 10190
    min_sell_amount_tokens = 1000

    trader = TokenTrader(rpc_url, wallet_address, private_key, token_address, usdt_address, eth_address, price_thresholds, lot_size_tokens, min_sell_amount_tokens)
    trader.trade()